In [1]:
import os
import sys
import json

curdir = os.getcwd()
while 'config.json' not in os.listdir(curdir):
    curdir = os.path.dirname(curdir)
with open(os.path.join(curdir,'config.json'), 'r') as f:
    dataset = json.load(f)['dataset']

curdir = os.getcwd()
while 'filepathhelper.py' not in os.listdir(curdir):
       curdir = os.path.dirname(curdir)
sys.path.append(curdir)
import filepathhelper

In [2]:
import math
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
# from tqdm import tqdm_notebook as tqdm
from neo4j import GraphDatabase
import neo4j
import json


In [3]:
def APSS(tx):
    result = tx.run("CALL algo.allShortestPaths.stream('weight',{nodeQuery:'Person',defaultValue:1.0}) "
                    "YIELD sourceNodeId, targetNodeId, distance "
                    "WITH sourceNodeId, targetNodeId, distance "
                    "WHERE algo.isFinite(distance) = true "
                    "MATCH (source:Person) WHERE id(source) = sourceNodeId "
                    "MATCH (target:Person) WHERE id(target) = targetNodeId "
                    "WITH source, target, distance WHERE source <> target "
                    "RETURN source.name AS source, target.name AS target, distance")
    return result


driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "jrdb"))
with driver.session() as session:
    sp = session.read_transaction(APSS)
driver.close()
PPsp = {}
for i in sp.records():
    u = i['source']
    v = i['target']
    w = i['distance']
    if u not in PPsp:
        PPsp[u] = {}
    PPsp[u][v] = w
with open(filepathhelper.path(dataset,'PPsp'),'wb') as f:
    pickle.dump(PPsp,f)

In [ ]:
def APSS(tx):
    result = tx.run("CALL algo.allShortestPaths.stream('weight',{nodeQuery:'Issue',defaultValue:1.0}) "
                    "YIELD sourceNodeId, targetNodeId, distance "
                    "WITH sourceNodeId, targetNodeId, distance "
                    "WHERE algo.isFinite(distance) = true "
                    "MATCH (source:Issue) WHERE id(source) = sourceNodeId "
                    "MATCH (target:Issue) WHERE id(target) = targetNodeId "
                    "WITH source, target, distance WHERE source <> target "
                    "RETURN source.name AS source, target.name AS target, distance")
    return result


driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "jrdb"))
with driver.session() as session:
    sp = session.read_transaction(APSS)
driver.close()
IIsp = {}
for i in sp.records():
    u = i['source']
    v = i['target']
    w = i['distance']
    if u not in IIsp:
        IIsp[u] = {}
    IIsp[u][v] = w
with open(filepathhelper.path(dataset,'IIsp'),'wb') as f:
    pickle.dump(IIsp,f)